# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id

where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-03-31'
'''

df_order = query(sql1)
df_order.shape

(2074218, 47)

In [5]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
, grade
from db_credit.risk
'''
df_risk = query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 提取订单风控数据

# 合并数据

In [8]:
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(2073907)

# 处理日期

In [9]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 渠道补充

In [10]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [11]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

# 渠道归属

In [12]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# 订单去重

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [14]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r', 'grade']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)

df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 


# 保留商家的数据

In [15]:
df_contain = df.copy()

# 剔除商家数据

In [16]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 去重订单明细导出

In [17]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-07-28'

# 导入服务订单

In [18]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(237761, 77)

In [19]:
df = clean.status_node(df)

# 导入台账数据

In [20]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
df_ck.shape

(26769, 18)

In [21]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(26687, 110)

In [23]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 将序列号转换为日期
base_date = pd.Timestamp('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


# 每日各渠道出库单数

In [38]:
# 按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

## 剔除拒量订单

In [25]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)

# 512分析

In [39]:
# 查看各个列的名称
# df.columns
# dfck['内存_y']
df_weekday_zh


归属渠道,京东渠道,其他渠道场景渠道。,单人聊天会话中的小程序消息卡片（分享）,小程序商家消息（服务提醒）,小程序快窗,我的小程序入口,抖音渠道,搜索渠道,支付宝 push 消息（同1014）,支付宝客户端首页,支付宝直播,支付宝社群,派金花,生活号,生活号-J-苹果旗舰,芝麻租物,订阅消息,All
日期,,,,,,,,,,,,,,,,,,
2025-01-01 00:00:00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2025-01-02 00:00:00,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,3
2025-01-03 00:00:00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3
2025-01-04 00:00:00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2
2025-01-05 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-24 00:00:00,1,0,0,0,0,0,0,9,0,0,0,0,0,0,0,4,0,14
2025-07-25 00:00:00,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,10,0,15
2025-07-26 00:00:00,1,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,0,7


In [36]:
dfck = dfck[dfck.内存_y.str.contains('512')==True]
df_grade_zh = pd.crosstab(dfck["grade"],dfck["归属渠道"],margins=True)
df_grade_zh

归属渠道,京东渠道,其他渠道场景渠道。,单人聊天会话中的小程序消息卡片（分享）,小程序商家消息（服务提醒）,小程序快窗,我的小程序入口,抖音渠道,搜索渠道,支付宝 push 消息（同1014）,支付宝客户端首页,支付宝直播,支付宝社群,派金花,生活号,生活号-J-苹果旗舰,芝麻租物,订阅消息,All
grade,,,,,,,,,,,,,,,,,,
A,9,1,2,1,0,5,1,181,0,1,0,0,0,1,1,204,0,407
B,12,1,0,1,0,2,0,70,0,1,1,0,0,0,2,117,0,207
C,9,0,0,0,0,1,1,36,0,0,2,0,1,0,0,60,0,110
D,0,0,0,0,1,0,1,2,0,0,0,0,0,0,0,20,0,24
E,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
F,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,3
S,13,2,4,0,0,5,0,345,1,4,2,1,0,1,0,314,1,693
All,43,4,6,2,1,13,5,635,1,6,5,1,1,2,3,716,1,1445


In [35]:
df_512 = df[df.内存.str.contains('512')==True]
df2_512 = df2[df2.内存.str.contains('512')==True]
df512_group = all_models.data_group(df_512, df2_512, df_risk_examine, 'grade')
df512_group['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df512_group[["机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '出库（按转化时间）', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
# "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",

,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
grade,,,,,,,,,,,,,,,,
A,2055.0,795,1260.0,38.84%,490,125,232.0,8,405,407,12.48%,15.10%,24.51%,7.15%,3.85%,12.48%
B,3371.0,1523,1848.0,20.40%,964,232,432.0,15,205,207,2.26%,10.64%,16.81%,4.77%,2.56%,2.26%
C,2132.0,970,1162.0,12.87%,574,127,344.0,9,108,110,1.20%,6.36%,10.75%,3.81%,1.41%,1.20%
D,755.0,275,480.0,5.57%,223,41,190.0,3,23,24,0.27%,2.59%,3.19%,2.21%,0.48%,0.27%
E,1.0,0,1.0,0.17%,1,0,0.0,0,0,1,0.00%,0.17%,0.00%,0.00%,0.00%,0.00%
F,9.0,5,4.0,0.13%,2,0,0.0,0,2,3,0.06%,0.06%,0.16%,0.00%,0.00%,0.06%
S,3117.0,1076,2041.0,49.98%,757,171,400.0,18,695,693,17.02%,18.54%,26.35%,9.79%,4.19%,17.02%
